# CD008 Ex02 - Gráficos com a API Altair

Equipe:
* Antonio Alisio de Meneses Cordeiro
* Jeferson Jose de Miranda
* Salvador Vicente Grisolia

# Dataset

Este caderno está focado no dataset de mortes sobre a série de TV Game of Thrones. O enredo fictício dessa série trata da dispuda de poder entre várias famílias e outros agrupamentos ou alianças que nesse caderno vamos chamar de facções. Devido à natureza dessa disputa, várias baixas vão ocorrer ao longo dos episódios. As perguntas que podem surgir nesse sentido envolvem: Qual o personagem que mais causou mortes? Qual foi o método mais utilizado para este fim? Qual casa/facção sofreu mais baixas? Vamos fazer uma análise com o auxílio da biblioteca Altair.

Disponível em: https://data.world/makeovermonday/2019w27

In [1]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Download do dataset
    import gdown
    url = 'https://drive.google.com/uc?id=1Eq29Li01ivCukkgOKLbxJYG3RQXG6eL7'
    gdown.download(url, 'deaths.csv')
    # Instalação da biblioteca Altair
    !pip install altair

In [2]:
import pandas as pd

deaths_df = pd.read_csv('deaths.csv')
deaths_df

,name,allegiance,season,episode,location,killer,killers_house,method,death_no
0,Waymar Royce,Night's Watch,1,1,Beyond the Wall,White Walker,None,Ice sword,1
1,Gared,Night's Watch,1,1,Beyond the Wall,White Walker,None,Ice sword,2
2,Will,Night's Watch,1,1,Winterfell,Ned Stark,House Stark,Sword,3
3,Stag,None,1,1,Winterfell,Direwolf,None,Teeth,4
4,Direwolf,None,1,1,Winterfell,Stag,None,Antler,5
...,...,...,...,...,...,...,...,...,...
2219,"Gregor ""the Mountain"" Clegane",House Lannister,8,5,King's Landing,"Sandor ""the Hound"" Clegane",House Clegane,Dragonfire (Dragon),2220
2220,"Sandor ""the Hound"" Clegane",House Lannister,8,5,King's Landing,"Sandor ""the Hound"" Clegane",House Clegane,Dragonfire (Dragon),2221
2221,Jaime Lannister,House Lannister,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Rubble,2222
2222,Cersei Lannister,House Lannister,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Rubble,2223


Nesse dataset temos os nome de quem morreu, sua família/facção, a temporada, o número do episódio, o assassino, a família/facção do assassino, o método do assassinato e a contagem ordenada da morte. Vemos que alguns valoers não estão definidos, por conta do indivíduo não ter uma facção definida, vamos resolver isso substituindo NaN por string vazia.

In [3]:
deaths_df = deaths_df.fillna('')
deaths_df

,name,allegiance,season,episode,location,killer,killers_house,method,death_no
0,Waymar Royce,Night's Watch,1,1,Beyond the Wall,White Walker,None,Ice sword,1
1,Gared,Night's Watch,1,1,Beyond the Wall,White Walker,None,Ice sword,2
2,Will,Night's Watch,1,1,Winterfell,Ned Stark,House Stark,Sword,3
3,Stag,None,1,1,Winterfell,Direwolf,None,Teeth,4
4,Direwolf,None,1,1,Winterfell,Stag,None,Antler,5
...,...,...,...,...,...,...,...,...,...
2219,"Gregor ""the Mountain"" Clegane",House Lannister,8,5,King's Landing,"Sandor ""the Hound"" Clegane",House Clegane,Dragonfire (Dragon),2220
2220,"Sandor ""the Hound"" Clegane",House Lannister,8,5,King's Landing,"Sandor ""the Hound"" Clegane",House Clegane,Dragonfire (Dragon),2221
2221,Jaime Lannister,House Lannister,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Rubble,2222
2222,Cersei Lannister,House Lannister,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Rubble,2223


# Qual a casa/facção que mais causou mortes?

In [5]:
import altair as alt
# Definindo uma paleta de cores personalizada com 100 cores distintas
custom_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
                  '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5',
                  '#393b79', '#7b4173', '#ff0000', '#0000ff', '#00ff00', '#ffff00', '#ff00ff', '#00ffff', '#800000', '#008000',
                  '#000080', '#800080', '#008080', '#808000', '#ff4500', '#228b22', '#0000cd', '#ffc0cb', '#dda0dd', '#b0c4de',
                  '#ffffe0', '#f0fff0', '#f0f8ff', '#f5f5f5', '#faebd7', '#7fffd4', '#f0ffff', '#f5deb3', '#adff2f', '#ff6347',
                  '#ffe4e1', '#da70d6', '#d3d3d3', '#90ee90', '#ff00ff', '#ffd700', '#a52a2a', '#20b2aa', '#778899', '#b0e0e6',
                  '#800080', '#a9a9a9', '#000000', '#006400', '#f08080', '#808080', '#add8e6', '#f08080', '#008000', '#ffdab9',
                  '#d3d3d3', '#ff0000', '#daa520', '#808080', '#c0c0c0', '#800000', '#ffff00', '#90ee90', '#ffd700', '#d2b48c',
                  '#f5f5f5', '#ffa500', '#ffdab9', '#d3d3d3', '#8b4513', '#ffffff', '#c0c0c0', '#ff00ff', '#800000', '#808000',
                  '#ffa500', '#ffff00', '#008000', '#00ff00', '#000080', '#008080', '#0000ff', '#00ffff', '#800080', '#ff0000',
                  '#ff00ff', '#008080', '#00ff00', '#000080', '#008080', '#0000ff', '#00ffff', '#800080', '#ff0000', '#ff00ff',
                  '#008080', '#00ff00', '#000080', '#008080', '#0000ff', '#00ffff', '#800080', '#ff0000', '#ff00ff', '#008080',
                  '#00ff00', '#000080', '#008080', '#0000ff', '#00ffff', '#800080', '#ff0000', '#ff00ff', '#008080', '#00ff00',
                  '#000080', '#008080', '#0000ff', '#00ffff', '#800080', '#ff0000', '#ff00ff', '#008080', '#00ff00', '#000080',
                  '#008080', '#0000ff', '#00ffff', '#800080']

# Definindo o número de cores da paleta
num_colors = len(custom_palette)

# Usando a paleta de cores personalizada
color_scale = alt.Scale(domain=list(range(num_colors)), range=custom_palette)

# Definindo o esquema de cores com a paleta personalizada
color_scheme = alt.Color('method:N', scale=color_scale)

In [6]:
len(custom_palette)

144

In [7]:
import altair as alt

def plotKillerMethod(df, killer_type):
    # Calcular o número de mortes por casa e método
    death_counts = df.groupby([killer_type, 'method']).size().reset_index(name='count')

    # Ordenar os dados pelo número de mortes
    death_counts_sorted = death_counts.sort_values(by='count', ascending=False)

    chart = alt.Chart(death_counts_sorted).mark_bar().encode(
        x=alt.X('count:Q', stack='normalize', axis=alt.Axis(title='Proporção')),
        y=alt.Y(f'{killer_type}:N', sort='-x', axis=alt.Axis(title='Casa do Assassino')),
        # color=alt.Color('method:N', scale=alt.Scale(scheme=color_scheme)),
        color=alt.Color('method:N', scale=alt.Scale(range=custom_palette)),
        tooltip=['method', 'count']
    ).properties(
        width=600,
        title=f'Causador da Morte ({killer_type}) e método (Ordenado por Número de Mortes)'
    )

    # Exibir o gráfico
    return chart

plotKillerMethod(deaths_df, 'killers_house')

alt.Chart(...)

Qualquer um que tenha acompanhado a série sabe o estrago que um dragão consegue fazer. Daí temos a casa Targaryen em primeiro lugar com o maior número de mortes causadas por fogo do dragão. Podemos plotar um gráfico de barras que mostra especificamente quais alianças foram afetadas:

In [8]:
df_temp = deaths_df[(deaths_df['killers_house']=='House Targaryen') & (deaths_df['method']=='Dragonfire (Dragon)')]
df_temp

,name,allegiance,season,episode,location,killer,killers_house,method,death_no
184,Pyat Pree,Warlocks of Qarth,2,10,Qarth,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),185
200,Kraznys mo Nakloz,Good Masters,3,4,Astapor,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),201
319,Goat,None,4,6,Meereen,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),320
439,Zalla,Smallfolk,4,10,Meereen,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),440
507,Great Master,House Targaryen,5,5,Meereen,Viserion,House Targaryen,Dragonfire (Dragon),508
...,...,...,...,...,...,...,...,...,...
2205,King's Landing Citizen,None,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),2206
2206,King's Landing Citizen,None,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),2207
2207,King's Landing Citizen,None,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),2208
2208,King's Landing Citizen,None,8,5,King's Landing,Daenerys Targaryen,House Targaryen,Dragonfire (Dragon),2209


In [9]:
def deathsByAllegiance(df):
    # Calcular a contagem de alianças por temporada
    counts = df.groupby(['season', 'allegiance']).size().reset_index(name='count')

    # Criar o gráfico de barras empilhadas horizontal com interatividade
    bars = alt.Chart(counts).mark_bar().encode(
        y=alt.Y('season:N', axis=alt.Axis(title='Temporada')),
        x=alt.X('sum(count):Q', axis=alt.Axis(title='Quantidade de Alianças')),
        color=alt.Color('allegiance:N', legend=alt.Legend(title='Aliança')),
        tooltip=['season:N', 'allegiance:N', 'count:Q'],
        row='name:N'
    ).properties(
        width=1000,
        height=400,
        title='Quantidade de Alianças por Temporada'
    ).interactive()

    # Exibir o gráfico
    return bars

deathsByAllegiance(df_temp)

alt.Chart(...)

E assim vemos que poder dos dragões foi utilizado mais nas últimas temporadas.

## Qual o personagem que, individualmente, causou mais mortes?

In [10]:
plotKillerMethod(deaths_df, 'killer')

alt.Chart(...)

Aqui temos um grande destaque Cersei Lannister que matou um grande número de pessoas com "fogo selvagem" em um armadilha que ocorreu na sexta temporada, matando um grande número de cidadãos de King's Landing e quase a totalidade dos militantes da fé.

In [11]:
df_temp = deaths_df[(deaths_df['killer']=='Cersei Lannister') & (deaths_df['method']=='Wildfire')]
df_temp

,name,allegiance,season,episode,location,killer,killers_house,method,death_no
796,Lancel Lannister,Faith Militant,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,797
797,High Sparrow,Faith Militant,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,798
798,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,799
799,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,800
800,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,801
...,...,...,...,...,...,...,...,...,...
989,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,990
990,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,991
991,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,992
992,King's Landing Noble,None,6,10,King's Landing,Cersei Lannister,House Lannister,Wildfire,993


In [12]:
deathsByAllegiance(df_temp)

alt.Chart(...)

# Quem matou quem, usando qual método?

In [13]:
def plotScatterKill(df):
    # Calcular o número de assassinatos por combinação de 'killer', 'method' e 'location'
    counts = df.groupby(['name', 'killer', 'method', 'location']).size().reset_index(name='count')

    # Determinar o tamanho máximo e mínimo de contagem
    max_count = counts['count'].max()
    min_count = counts['count'].min()

    # Definir limites da escala de tamanho
    min_size = 50  # Tamanho mínimo
    max_size = 200  # Tamanho máximo

    # Criar o gráfico de dispersão com escala ajustada
    scatter = alt.Chart(counts).mark_circle().encode(
        x=alt.X('killer:N', axis=alt.Axis(title='Assassino')),
        y=alt.Y('name:N', axis=alt.Axis(title='Vítima')),
        color='location:N',
        size=alt.Size('count:Q',
                      scale=alt.Scale(domain=[min_count, max_count], range=[min_size, max_size]),
                      legend=alt.Legend(title='Contagem')),
        tooltip=['name', 'killer', 'method', 'location', 'count']
    ).properties(
        width=1200,
        height=2000,
        title='Relação entre Vítima, Assassino, Método e Localização'
    ).configure_axis(
        grid=True
    ).interactive()

    # Exibir o gráfico
    return scatter

plotScatterKill(deaths_df)

alt.Chart(...)

# Qual família/facção sofreu mais baixas ao longo das temporadas?

In [14]:
def deathsByAllegiance(df):
    # Calcular a contagem de alianças por temporada
    counts = df.groupby(['season', 'allegiance']).size().reset_index(name='count')

    # Criar o gráfico de barras empilhadas horizontal com interatividade
    bars = alt.Chart(counts).mark_bar().encode(
        y=alt.Y('season:N', axis=alt.Axis(title='Temporada')),
        x=alt.X('sum(count):Q', axis=alt.Axis(title='Quantidade de Alianças')),
        color=alt.Color('allegiance:N', legend=alt.Legend(title='Aliança')),
        tooltip=['season:N', 'allegiance:N', 'count:Q'],
        row='name:N'
    ).properties(
        width=1000,
        height=400,
        title='Quantidade de Alianças por Temporada'
    ).interactive()

    # Exibir o gráfico
    return bars

deathsByAllegiance(deaths_df)

alt.Chart(...)

Fazendo uma análise de quem mais sofreu baixas, por temporada, com o base no gráfico, vamos relembrar algumas tragédias:
Primeira temporada: Grandes nomes da casa Stark, e alguns soldados.

In [15]:
df_temp = deaths_df[
    (deaths_df['season']==1) &
    (deaths_df['allegiance']=='House Stark')
]
df_temp['name'].unique()

array(['Stark soldier', 'Jory Cassel', 'Lady', 'Vayon Poole',
       'Stark staff member', 'Septa Mordane', 'Syrio Forel', 'Ned Stark'],
      dtype=object)

Segunda temporada: Casa Baratheon do lado do irmão Stannis.

In [16]:
df_temp = deaths_df[
    (deaths_df['season']==2) &
    (deaths_df['allegiance']=='House Baratheon of Dragonstone')
]
df_temp['name'].unique()

array(['Maester Cressen', 'Matthos Seaworth',
       'Baratheon of Dragonstone soldier'], dtype=object)

Terceira temporada: Novo infortúneo da casa Stark. Contando com "um dos", senão "o" episódio mais chocante da série.

In [17]:
df_temp = deaths_df[
    (deaths_df['season']==3) &
    (deaths_df['allegiance']=='House Stark')
]
df_temp['name'].unique()

array(['Stark soldier', 'Rickard Karstark', 'Unborn Stark child',
       'Talisa Stark', 'Grey Wind', 'Robb Stark', 'Catelyn Stark'],
      dtype=object)

Quarta temporada: Grandes baixas dos homens livres que viviam além da muralha, na batalha contra a patrulha da noite.

In [18]:
df_temp = deaths_df[
    (deaths_df['season']==4) &
    (deaths_df['allegiance']=='Free Folk')
]
df_temp['name'].unique()

array(['Wildling', 'Thenn raider', 'Giant wildling',
       'Mag the Mighty giant', 'Styr', 'Ygritte'], dtype=object)

Quinta temporada: Baixas concentradas nos "Sons of the Harpy", no embate contra as forças de Daenerys Targaryen.

In [19]:
df_temp = deaths_df[
    (deaths_df['season']==5) &
    (deaths_df['allegiance']=='Sons of the Harpy')
]
df_temp['name'].unique()

array(['Sons of the Harpy agent'], dtype=object)

Sexta temporada:

In [20]:
df_temp = deaths_df[
    (deaths_df['season']==6) &
    (deaths_df['allegiance']=='')
]
df_temp['name'].unique()

array([], dtype=object)

Baixa de...

In [21]:
((deaths_df['season'] == 6) & (deaths_df['name'] == "Meereen citizen")).sum()

14

...cidadãos de Meereen, em confronto contra as forças de Daenerys e de...

In [22]:
((deaths_df['season'] == 6) & (deaths_df['name'] == "King's Landing Noble")).sum()

138

nobres de "King's Landing".

Sétima temporada: Muitos baixas no exército Lannister na batalha contra os dragões de Daenerys, incluindo execução por fogo do dragão dos chefe da família Tarly e seu filho.

In [23]:
df_temp = deaths_df[
    (deaths_df['season']==7) &
    (deaths_df['allegiance']=='House Lannister')
]
df_temp['name'].unique()

array(['Lannister soldier', 'Horse', 'Randyll Tarly', 'Dickon Tarly'],
      dtype=object)

Oitava temporada: Queda de "King's Landing" resultando na morte dos principais nomes da casa Lannister.

In [24]:
df_temp = deaths_df[
    (deaths_df['season']==8) &
    (deaths_df['allegiance']=='House Lannister')
]
df_temp['name'].unique()

array(['Lannister soldier', 'Golden Company soldier',
       'Golden Company horse', 'Harry Strickland', 'Qyburn',
       'Gregor "the Mountain" Clegane', 'Sandor "the Hound" Clegane',
       'Jaime Lannister', 'Cersei Lannister'], dtype=object)